In [86]:
import sqlite3
import pandas as pd

In [87]:
con = sqlite3.connect(':memory:')

In [88]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [89]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [90]:
pd.read_sql(
    '''
    select * from Products
    ''',
    con,
)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18
1,2,Chang,1,1,24 - 12 oz bottles,19
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15
73,74,Longlife Tofu,4,7,5 kg pkg.,10
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,июль.75
75,76,Lakkalikööri,23,1,500 ml,18


In [91]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [92]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName as Name, Country, Address FROM Customers
    WHERE Country IN ('Germany','France') OR City ='Madrid'
    ''',
    con,
)

,Name,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [93]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country,COUNT(CustomerName) AS Quantity FROM Customers
    GROUP BY Country
    ORDER BY Quantity DESC
    LIMIT 3
    ''',
    con,
)

,Country,Quantity
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [94]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName as Name, OrderDate FROM Orders
    JOIN Shippers ON Shippers.ShipperID = Orders.ShipperID
    ORDER BY OrderDate
    LIMIT 1 OFFSET 9
    ''',
    con,
)

,Name,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [95]:
tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, Quantity, Price, Quantity*Price as Summa from Products
    JOIN OrderDetails ON Products.ProductID = OrderDetails.ProductID
    JOIN Orders ON OrderDetails.OrderID = Orders.OrderID
    WHERE Orders.OrderID = (select OrderID from (select Orders.OrderID,SUM(Quantity*Price) as QW FROM Orders
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    JOIN Products ON Products.ProductID = OrderDetails.ProductID
    GROUP BY Orders.OrderID
    Order by QW desc
    limit 1))
    ''',
    con,
)

,ProductName,Quantity,Price,Summa
0,Sir Rodney's Marmalade,12,81,972.0
1,Côte de Blaye,40,263.5,10540.0
2,Camembert Pierrot,70,34,2380.0
3,Mozzarella di Giovanni,42,34.8,1461.6


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [96]:
tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, Quantity FROM OrderDetails
    JOIN Products ON Products.ProductID = OrderDetails.ProductID
    WHERE ProductName = (SELECT ProductName FROM(SELECT ProductName, sum(Quantity) AS SUMMA from OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    GROUP BY ProductName
    ORDER BY SUMMA desc
    LIMIT 1))
    

    ''',
    con,
)

,ProductName,Quantity
0,Gorgonzola Telino,4
1,Gorgonzola Telino,14
2,Gorgonzola Telino,15
3,Gorgonzola Telino,20
4,Gorgonzola Telino,20
5,Gorgonzola Telino,25
6,Gorgonzola Telino,30
7,Gorgonzola Telino,30
8,Gorgonzola Telino,32
9,Gorgonzola Telino,40


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [97]:
tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName, Country, ContactName, Phone FROM Orders
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    JOIN Suppliers ON Products.SupplierID = Suppliers.SupplierID
    GROUP BY SupplierName
    ORDER BY COUNT(Products.SupplierID) desc
    LIMIT 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [98]:
tasks+=1
pd.read_sql(
    '''
    SELECT Customers.Country, Categories.CategoryName, SUM(Quantity*Price) as SUMMA FROM Orders
    JOIN Customers ON Orders.CustomerID = Customers.CustomerID
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    JOIN Products ON OrderDetails.ProductID =Products.ProductID
    JOIN Categories ON Products.CategoryID = Categories.CategoryID
    WHERE Customers.Country = 'Brazil'
    GROUP BY Products.CategoryID
    ORDER BY SUMMA desc
    LIMIT 1
    ''',
    con,
)

,Country,CategoryName,SUMMA
0,Brazil,Beverages,13636.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [99]:
tasks+=1
pd.read_sql(
    '''
    SELECT MAX(SUMMA) - MIN(SUMMA) as ITOG FROM (
    SELECT SUM(Quantity*Price) as SUMMA FROM Orders
    JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    JOIN Products ON Products.ProductID = OrderDetails.ProductID
    JOIN Customers ON Orders.CustomerID =Customers.CustomerID
    WHERE Customers.Country = 'USA'
    GROUP BY Orders.OrderID
    )
    
    ''',
    con,
)

,ITOG
0,7698.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [100]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT(Orders.EmployeeID) as COUNT_ORDER, (FirstName || ' '|| LastName) as FULL_NAME FROM Orders
    JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
    WHERE Orders.EmployeeID IN (SELECT EmployeeID FROM (SELECT  *, (substr(BirthDate, 7, 4) || '-' || substr(BirthDate, 4, 2) || '-' || substr(BirthDate, 1, 2)) as new_date
    FROM Employees 
    ORDER BY new_date desc
    LIMIT 3))
    GROUP BY Orders.EmployeeID
    ''',
    con,
)

,COUNT_ORDER,FULL_NAME
0,29,Nancy Davolio
1,31,Janet Leverling
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [101]:
tasks+=1
pd.read_sql(
    '''
    SELECT SUM(Quantity)*24 as ITOG_Crab_Meat FROM OrderDetails
    JOIN Products ON OrderDetails.ProductID = Products.ProductID
    WHERE Products.ProductID = (SELECT ProductID FROM Products WHERE ProductName LIKE '%Crab Meat')
   
    ''',
    con,
)

,ITOG_Crab_Meat
0,6144


In [102]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
